!pip install torchsummary
!pip install torchview

In [1]:
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('fivethirtyeight') 
%matplotlib inline


# import librosa for analysing audio signals : visualize audio, display the spectogram
import librosa
import soundfile as sf
# import librosa for analysing audio signals : visualize audio, display the spectogram
import librosa.display
# import wav for reading and writing wav files
import wave

# import os for system operations
import os

# import random for get random values/choices
import random


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from torchview import draw_graph

from pathlib import Path
import torchaudio
print("Libraries imported - ready to use PyTorch", torch.__version__)

# import tqdm to show a smart progress meter
from tqdm.notebook import trange,tqdm

# import warnings to hide the unnessairy warniings
import warnings
warnings.filterwarnings('ignore')

Libraries imported - ready to use PyTorch 2.2.2


In [2]:
import random 
SEED = 42
def seed_everything(seed=42): 
    random.seed(seed) 
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed) 
seed_everything(SEED)

In [3]:
%ls ../../0_DATASETS/kaggle/input/environmental-sound-classification-50

audio/  bc_utils.py  esc50.csv  utils2.py  utils.py


In [4]:
import os
print(os.getcwd())
# To get a list of the paths of all the audio files
path = '../../0_DATASETS/kaggle/input/environmental-sound-classification-50'
data = "../../0_DATASETS/kaggle/input/environmental-sound-classification-50/audio/audio/44100/"
dataset = pd.read_csv(os.path.join(path, 'esc50.csv'))
all_files = []
for path, subdirs, files in os.walk(data):
    for name in files:
        all_files.append(os.path.join(path, name))
        
# A dictionary to decode the categories into targets
decoder = {0: 'dog', 14: 'chirping_birds', 36: 'vacuum_cleaner', 19: 'thunderstorm', 30: 'door_wood_knock',34: 'can_opening', 9: 'crow', 22: 'clapping', 48: 'fireworks', 41: 'chainsaw', 47: 'airplane', 31: 'mouse_click', 17: 'pouring_water', 45: 'train', 8: 'sheep', 15: 'water_drops', 46: 'church_bells', 37: 'clock_alarm', 32: 'keyboard_typing', 16: 'wind', 25: 'footsteps', 4: 'frog', 3: 'cow', 27: 'brushing_teeth', 43: 'car_horn', 12: 'crackling_fire', 40: 'helicopter', 29: 'drinking_sipping', 10: 'rain', 7: 'insects', 26: 'laughing', 6: 'hen', 44: 'engine', 23: 'breathing', 20: 'crying_baby', 49: 'hand_saw', 24: 'coughing', 39: 'glass_breaking', 28: 'snoring', 18: 'toilet_flush', 2: 'pig', 35: 'washing_machine', 38: 'clock_tick', 21: 'sneezing', 1: 'rooster', 11: 'sea_waves', 42: 'siren', 5: 'cat', 33: 'door_wood_creaks', 13: 'crickets'}

# A dictionary to encode the categories into targets
encoder = {'dog': 0, 'chirping_birds': 14, 'vacuum_cleaner': 36, 'thunderstorm': 19, 'door_wood_knock': 30, 'can_opening': 34, 'crow': 9, 'clapping': 22, 'fireworks': 48, 'chainsaw': 41, 'airplane': 47, 'mouse_click': 31, 'pouring_water': 17, 'train': 45, 'sheep': 8, 'water_drops': 15, 'church_bells': 46, 'clock_alarm': 37, 'keyboard_typing': 32, 'wind': 16, 'footsteps': 25, 'frog': 4, 'cow': 3, 'brushing_teeth': 27, 'car_horn': 43, 'crackling_fire': 12, 'helicopter': 40, 'drinking_sipping': 29, 'rain': 10, 'insects': 7, 'laughing': 26, 'hen': 6, 'engine': 44, 'breathing': 23, 'crying_baby': 20, 'hand_saw': 49, 'coughing': 24, 'glass_breaking': 39, 'snoring': 28, 'toilet_flush': 18, 'pig': 2, 'washing_machine': 35, 'clock_tick': 38, 'sneezing': 21, 'rooster': 1, 'sea_waves': 11, 'siren': 42, 'cat': 5, 'door_wood_creaks': 33, 'crickets': 13}

/home/mln/git_explore/kaggle/esc50_sound_classification


In [5]:
# Importing 1 file
y, sr = librosa.load(data + "1-100032-A-0.wav")

print('y:', y, '\n')
print('y shape:', np.shape(y), '\n')
print('Sample Rate (KHz):', sr, '\n')

# The duration is equal to the number of frames divided by the framerate
print('Duration of the audio file:', np.shape(y)[0]/sr, 'second')

y: [0. 0. 0. ... 0. 0. 0.] 

y shape: (110250,) 

Sample Rate (KHz): 22050 

Duration of the audio file: 5.0 second


In [6]:

path_audio = os.path.join(path, "/kaggle/input/environmental-sound-classification-50/audio/audio/16000/")

In [7]:
def add_noise(data):
    noise = np.random.normal(0, 0.1, len(data))
    audio_noisy = data + noise
    return audio_noisy
    
def pitch_shifting(data):
    sr  = 16000
    bins_per_octave = 12
    pitch_pm = 2
    pitch_change =  pitch_pm * 2*(np.random.uniform())   
    data = librosa.effects.pitch_shift(data.astype('float64'),  sr, n_steps=pitch_change, 
                                          bins_per_octave=bins_per_octave)
    return pitch_shifting

def random_shift(data):
    timeshift_fac = 0.2 *2*(np.random.uniform()-0.5)  # up to 20% of length
    start = int(data.shape[0] * timeshift_fac)
    if (start > 0):
        data = np.pad(data,(start,0),mode='constant')[0:data.shape[0]]
    else:
        data = np.pad(data,(0,-start),mode='constant')[0:data.shape[0]]
    return data

def volume_scaling(data):
    sr  = 16000
    dyn_change = np.random.uniform(low=1.5,high=2.5)
    data = data * dyn_change
    return data
    
def time_stretching(data, rate=1.5):
    input_length = len(data)
    streching = data.copy()
    streching = librosa.effects.time_stretch(streching, rate)
    
    if len(streching) > input_length:
        streching = streching[:input_length]
    else:
        streching = np.pad(data, (0, max(0, input_length - len(streching))), "constant")
    return streching
def audio_augmentation(file, aug):
    directory = 'ESC-50-augmented-data/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    aug = np.array(aug,dtype='float32').reshape(-1,1)
    sf.write(directory+'/'+ file, aug, 16000, 'PCM_24')

In [8]:
def data_aug():
    path_ = np.random.choice(os.listdir(path_audio), size = (2000,), replace= False)
    for k,files in zip(range(len(path_)), path_):
        if files[0] != "5":
            data_, fs = librosa.load(os.path.join(path_audio, files), sr = 16000)
            noise_data = add_noise(data_)
    #         pitch_data = pitch_shifting(data_)
            random_shift_data = random_shift(data_)
            volume_scale_data = volume_scaling(data_)
            time_stretching_data =  time_stretching(data_, rate=1.5)
            aug = [noise_data,time_stretching_data, random_shift_data,volume_scale_data ]
            for j in range(len(aug)):
                filename = (files[0:2]+'generated'+'-'+str(j)+'-'+str(k)+'-'+files[2:])
                audio_augmentation(filename, aug[j])

In [9]:
dataset['files_path'] = data+dataset['filename']
dataset  = dataset.drop(columns=['esc10','take','src_file','category'])
dataset.head()

,filename,fold,target,files_path
0,1-100032-A-0.wav,1,0,../../0_DATASETS/kaggle/input/environmental-so...
1,1-100038-A-14.wav,1,14,../../0_DATASETS/kaggle/input/environmental-so...
2,1-100210-A-36.wav,1,36,../../0_DATASETS/kaggle/input/environmental-so...
3,1-100210-B-36.wav,1,36,../../0_DATASETS/kaggle/input/environmental-so...
4,1-101296-A-19.wav,1,19,../../0_DATASETS/kaggle/input/environmental-so...


In [10]:
def update_dataframe():
    all_files = []
    data_aug = os.path.join(path, '/kaggle/working/ESC-50-augmented-data/')
    for path, subdirs, files in os.walk(data_aug):
        for name in files:
            fold = int(name[0])
            target = int(name.split('-')[-1].replace('.wav', ''))
            file_path = data_aug + name
            dataset.loc[-1] = [name, fold, target, file_path]
            dataset.index = dataset.index + 1  # shifting index
            dataset = dataset.sort_index()
            all_files.append(os.path.join(path, name))

In [11]:
def create_esc_dataset():
    n_samples = dataset['filename'].shape[0]
    dataset['zero_crossing_rate'] = np.zeros(n_samples)
    dataset['chroma_stft'] = np.zeros(n_samples)
    dataset['rmse'] = np.zeros(n_samples)
    dataset['spectral_centroid'] = np.zeros(n_samples)
    dataset['spectral_bandwidth'] = np.zeros(n_samples)
    dataset['beat_per_minute'] = np.zeros(n_samples)
    dataset['rolloff'] = np.zeros(n_samples)
    for i in range(0, 20):
        dataset[f'mfcc_{i}'] = np.zeros((n_samples,))

    for i in tqdm(range(n_samples)):
        y , sr = librosa.load(dataset['files_path'][i], mono=True)
        dataset['rmse'][i] = np.mean(librosa.feature.rms(y=y))
        dataset['chroma_stft'][i] = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
        dataset['spectral_centroid'][i] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        dataset['spectral_bandwidth'][i] = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        dataset['rolloff'][i] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        dataset['zero_crossing_rate'][i] = np.mean(librosa.feature.zero_crossing_rate(y))
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        dataset['beat_per_minute'] = tempo
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        for e in range(0, 20):
            dataset[f'mfcc_{e}'][i] = np.mean(mfcc[e])

path = '../../0_DATASETS/kaggle/input/esc-features-extracted'
dataset = pd.read_csv(os.path.join(path, 'Esc50_features_extracted.csv'))
dataset.head()

,Unnamed: 0,filename,fold,target,category,esc10,src_file,take,files_path,zero_crossing_rate,...,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19
0,0,1-100032-A-0.wav,1,0,dog,True,100032,A,/kaggle/input/environmental-sound-classificati...,0.011952,...,-0.676718,-0.050617,-1.051819,0.645590,-0.260393,-0.159993,-1.281796,0.069487,-0.281899,0.620376
1,1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,/kaggle/input/environmental-sound-classificati...,0.303564,...,-20.726339,-18.533272,-16.482960,-10.614421,-13.925037,-6.964433,-11.118065,1.713619,-8.622952,11.348335
2,2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,/kaggle/input/environmental-sound-classificati...,0.385378,...,-6.736873,19.633633,-3.546270,28.449459,-13.570659,11.713070,-2.816873,7.117002,0.840253,4.870029
3,3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,/kaggle/input/environmental-sound-classificati...,0.386841,...,-4.445976,18.834217,-3.205974,27.602684,-12.762237,11.530251,-2.151586,8.215571,1.618893,4.714815
4,4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,/kaggle/input/environmental-sound-classificati...,0.045880,...,-13.565428,4.434554,-15.572582,2.263079,-9.330681,-0.770855,-3.896152,-2.439542,-12.992370,0.887699


In [12]:
%ls ../../0_DATASETS/kaggle/input/esc-features-extracted/

Esc50_features_extracted.csv  model_1_HDA.pt  model_2_HDA.pt


In [13]:
train = dataset[dataset['fold'] < 5]
test = dataset[dataset['fold'] == 5]

X_train = train.drop(columns=['files_path','fold','filename','target' ,'src_file', 'take','category'])
y_train = train.target

X_test = test.drop(columns=['files_path','fold','filename','target' ,'src_file', 'take','category'])
y_test = test.target

print("Size of the training set : ", X_train.shape )
print("Size of the testing set : ", X_test.shape )

Size of the training set :  (1600, 29)
Size of the testing set :  (400, 29)


## B. Second Method :  Working on the Raw Audio

### 1. Data Loader

In [14]:
class DataGenerator(Dataset):
    def __init__(self, path, transform=None, kind='train'):
        if kind == 'train':
            files = Path(path).glob('[1-4]-*')
            self.items = [(str(file), file.name.split('-')[-1].replace('.wav', '')) for file in files]
        if kind == 'test':
            files = Path(path).glob('5-*')
            self.items = [(str(file), file.name.split('-')[-1].replace('.wav', '')) for file in files]

        self.length = len(self.items)
        print(self.length)

    def __getitem__(self, index):
        filename, label = self.items[index]
        data_tensor, rate = torchaudio.load(filename)
        tmp = data_tensor[0, 0:80000]
        return (tmp, int(label))

    def __len__(self):
        return self.length

### 2. Creating Train, Validation, Test sets

In [15]:
path_audio = '../../0_DATASETS/kaggle/input/environmental-sound-classification-50/audio/audio/16000/'
batch_size = 64

train_data = DataGenerator(path_audio, kind='train')
test_data = DataGenerator(path_audio, kind='test')

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

1600
400


In [16]:
%ls ../../0_DATASETS/kaggle/input/environmental-sound-classification-50/audio/audio/16000/

1-100032-A-0.wav   2-118624-A-30.wav  3-151212-A-24.wav  4-196671-A-8.wav
1-100038-A-14.wav  2-118625-A-30.wav  3-151213-A-24.wav  4-196671-B-8.wav
1-100210-A-36.wav  2-118817-A-32.wav  3-151255-A-28.wav  4-196672-A-8.wav
1-100210-B-36.wav  2-118964-A-0.wav   3-151269-A-35.wav  4-197103-A-6.wav
1-101296-A-19.wav  2-119102-A-21.wav  3-151273-A-35.wav  4-197454-A-28.wav
1-101296-B-19.wav  2-119139-A-31.wav  3-151557-A-28.wav  4-197454-B-28.wav
1-101336-A-30.wav  2-119161-A-8.wav   3-151557-B-28.wav  4-197871-A-15.wav
1-101404-A-34.wav  2-119161-B-8.wav   3-152007-A-20.wav  4-198025-A-23.wav
1-103298-A-9.wav   2-119161-C-8.wav   3-152007-B-20.wav  4-198360-A-49.wav
1-103995-A-30.wav  2-119748-A-38.wav  3-152007-C-20.wav  4-198360-B-49.wav
1-103999-A-30.wav  2-120218-A-30.wav  3-152007-D-20.wav  4-198360-C-49.wav
1-104089-A-22.wav  2-120218-B-30.wav  3-152007-E-20.wav  4-198841-A-37.wav
1-104089-B-22.wav  2-120333-A-32.wav  3-152020-A-36.wav  4-198962-A-25.wav
1-105224-A-22.wav  2-120586-A

### 3. Model 

This is a CNN (Convolutional Neural Network) architecture implemented using PyTorch's nn.Module class. The architecture is composed of several layers:

- The first layer is a 2D convolutional layer with 1 input channel, 16 output channels, kernel size of (1,8), stride of (1,1) and padding "same".
- The second layer is a batch normalization layer.
- The third layer is another 2D convolutional layer with 16 input channels and 16 output channels, kernel size of (1,8), stride of (1,1) and padding "same".
- The fourth layer is another batch normalization layer.
- The fifth layer is a max pooling layer with kernel size of (1,128), stride of (1,128) and no padding.
- The sixth layer is a 2D convolutional layer with 1 input channel, 32 output channels, kernel size of (3,3), stride of (1,1) and padding of 1.
- The seventh layer is a batch normalization layer.
- The eight layer is another 2D convolutional layer with 32 input channels and 32 output channels, kernel size of (3,3), stride of (1,1) and padding of 1.
- The nineth layer is another batch normalization layer.
- The tenth layer is a max pooling layer with kernel size of 4 and no padding.
- The eleventh layer is a 2D convolutional layer with 32 input channels, 64 output channels, kernel size of (3,3), stride of (2,2) and padding of 2.
- The twelfth layer is a batch normalization layer.
- The thirteenth layer is another 2D convolutional layer with 64 input channels and 64 output channels, kernel size of (3,3), stride of (2,2) and padding of 1.
- The fourteenth layer is another batch normalization layer.
- The fifteenth layer is a max pooling layer with kernel size of 2 and no padding.
- The sixteenth layer is a 2D convolutional layer with 64 input channels, 128 output channels, kernel size of (3,3), stride of (2,2) and padding of 1.
- The seventeenth layer is a batch normalization layer.
- The eighteenth layer is another 2D convolutional layer with 128 input channels and 128 output channels, kernel size of (3,3), stride of (2,2) and padding of 1.
- The nineteenth layer is another batch normalization layer.
- The twentieth layer is a max pooling layer with kernel size of (1,2) and no padding.
- The last layers are a fully connected linear layer with 256 input features and num_classes output features, and a dropout layer with a probability of 0.2.

<div class="alert alert-block alert-info" style="font-size:14px; color:black; background-color:#ffffcc; border-color: #e6e600">
    <aside>
📒
Each layer in the architecture is designed to extract relevant features from the input audio signal and pass them on to the next layer. The convolutional layers with different kernel sizes and strides are responsible for extracting features at different scales and locations in the signal. The max pooling layers reduce the spatial resolution of the feature maps and help to reduce the number of parameters in the network while also making it robust to small translations in the input signal.
        

The fully connected linear layer and the dropout layer at the end of the architecture are used to produce the final classification of the audio signal into one of the 50 classes.

In [17]:
# Create a neural net class
class Net(nn.Module):
    
    
    # Defining the Constructor
    def __init__(self, num_classes=50):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(1,8), stride=(1,1), padding="same")
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(1,8), stride=(1,1), padding="same")
        self.bn2 = nn.BatchNorm2d(16)
        
        self.pool_1 = nn.MaxPool2d(kernel_size=(1,128), stride = (1,128), padding=0) 
        
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), stride=(1,1), padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,3), stride=(1,1), padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        
        self.pool_2 = nn.MaxPool2d(kernel_size=4, padding=0) 
            
        self.conv5 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=(2,2), padding=2)
        self.bn5 = nn.BatchNorm2d(64)
        self.conv6 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(2,2), padding=1)
        self.bn6 = nn.BatchNorm2d(64)
        
        self.pool_3 = nn.MaxPool2d(kernel_size=2, padding=0) 
        
        self.conv7 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=(2,2), padding=1)
        self.bn7 = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=(2,2), padding=1)
        self.bn8 = nn.BatchNorm2d(128)
        
        self.pool_4 = nn.MaxPool2d(kernel_size=(1,2), padding=0) 
        
        self.dense = nn.Linear(in_features= 256, out_features=num_classes)
        # Define proportion or neurons to dropout
        self.dropout = nn.Dropout(0.2)
        
        

    def forward(self, x):
        x = x.unsqueeze(1).view(-1, 1, 1, 80000)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool_1(x)
        x = x.view((-1,1,16, 625))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool_2(x)
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.dropout(x)
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool_3(x)
        x = F.relu(self.bn7(self.conv7(x)))
        x = self.dropout(x)
        x = F.relu(self.bn8(self.conv8(x)))
        x = self.pool_4(x)
        x = x.view(x.size(0),-1)
        x = self.dense(x)
        x = self.dropout(x)
        return x
    
device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"

In [18]:
model = Net(num_classes=50).to(device)
input_data = torch.randn(64, 1, 80000)
model_graph = draw_graph(model,input_data, roll=True)
#model_graph.visual_graph

In [19]:
# Create an instance of the model class and allocate it to the device
summary(model,(1,80000))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 1, 80000]             144
       BatchNorm2d-2         [-1, 16, 1, 80000]              32
           Dropout-3         [-1, 16, 1, 80000]               0
            Conv2d-4         [-1, 16, 1, 80000]           2,064
       BatchNorm2d-5         [-1, 16, 1, 80000]              32
         MaxPool2d-6           [-1, 16, 1, 625]               0
            Conv2d-7          [-1, 32, 16, 625]             320
       BatchNorm2d-8          [-1, 32, 16, 625]              64
           Dropout-9          [-1, 32, 16, 625]               0
           Conv2d-10          [-1, 32, 16, 625]           9,248
      BatchNorm2d-11          [-1, 32, 16, 625]              64
        MaxPool2d-12           [-1, 32, 4, 156]               0
           Conv2d-13            [-1, 64, 3, 79]          18,496
      BatchNorm2d-14            [-1, 64

### 4. Training Loop

### 6. Training & Evaluating the model

In [20]:
import torch

if torch.cuda.is_available():
    print("CUDA GPU is available.")
    device = torch.device("cuda")
    print("Device:", device)
    print("Number of GPUs:", torch.cuda.device_count())
    print("Current GPU:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA GPU is not available. Using CPU.")
    device = torch.device("cpu")
    print("Device:", device)

CUDA GPU is available.
Device: cuda
Number of GPUs: 1
Current GPU: 0
GPU Name: NVIDIA GeForce RTX 4060 Ti


In [21]:
import torch
from tqdm import tqdm

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    for batch_idx, (data, target) in enumerate(pbar):
        # Move data and target to the device
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_criteria(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss=loss.item())
    return loss.item()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        pbar = tqdm(test_loader, desc="Testing")
        for data, target in pbar:
            # Move data and target to the device
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_criteria(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            pbar.set_postfix(loss=(test_loss / len(test_loader)), accuracy=(correct / len(test_loader.dataset)))

    test_loss /= len(test_loader.dataset)
    return test_loss

def training(model):
    # Use an "Adam" optimizer to adjust weights
    optimizer = optim.Adam(model.parameters(), lr=3e-4)

    # Specify the loss criteria
    loss_criteria = nn.CrossEntropyLoss()

    # Track metrics in these arrays
    epoch_nums = []
    training_loss = []
    validation_loss = []

    # Train over 10 epochs (We restrict to 10 for time issues)
    epochs = 120
    print('Training on', device)
    for epoch in range(1, epochs + 1):
        train_loss = train(model, device, train_loader, optimizer, epoch)
        test_loss = test(model, device, test_loader)
        epoch_nums.append(epoch)
        training_loss.append(train_loss)
        validation_loss.append(test_loss)

# Move the model to the device
model = model.to(device)

loss_criteria = nn.CrossEntropyLoss()
training(model)  # Complete the training process

# Put the model in evaluation mode
model = model.eval()

# Save the entire model
torch.save(model, 'trained_model.pt')



Training on cuda


Epoch 1:   4%|▍         | 1/25 [00:00<00:13,  1.75it/s, loss=4.11]

Testing: 100%|██████████| 7/7 [00:02<00:00,  3.34it/s, accuracy=0.465, loss=2.1]  
